# Split Data Set

In [190]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer

In [191]:
df = pd.read_csv('../data/processed/preprocessed_data.csv')

In [192]:
df["Day"] = pd.to_datetime(df["Day"], format='%Y-%m-%d')
df["Month"] = pd.to_datetime(df["Day"]).dt.month
df['Year'] = pd.to_datetime(df['Day']).dt.year

# I dropped the "Day" column because it was causing an error.
df = df.drop("Day", axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 198900 entries, 0 to 198899
Columns: 199 entries, Year to Entity_Zimbabwe
dtypes: float64(197), int32(2)
memory usage: 300.5 MB


In [193]:
df.head(20)

,Year,Daily Average Temp,Monthly Average Temp,Month,Entity_Afghanistan,Entity_Albania,Entity_Algeria,Entity_American Samoa,Entity_Andorra,Entity_Angola,...,Entity_United States Virgin Islands,Entity_Uruguay,Entity_Uzbekistan,Entity_Vanuatu,Entity_Venezuela,Entity_Vietnam,Entity_World,Entity_Yemen,Entity_Zambia,Entity_Zimbabwe
0,1940,-2.032494,11.327695,1,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1940,-0.733503,11.327695,2,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1940,1.999134,11.327695,3,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1940,10.199754,11.327695,4,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1940,17.942135,11.327695,5,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,1940,23.203533,11.327695,6,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,1940,24.441550,11.327695,7,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,1940,22.058195,11.327695,8,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,1940,17.721355,11.327695,9,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,1940,13.730240,11.327695,10,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [194]:
X = df.drop('Daily Average Temp', axis=1)
y = df['Daily Average Temp']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train

,Year,Monthly Average Temp,Month,Entity_Afghanistan,Entity_Albania,Entity_Algeria,Entity_American Samoa,Entity_Andorra,Entity_Angola,Entity_Anguilla,...,Entity_United States Virgin Islands,Entity_Uruguay,Entity_Uzbekistan,Entity_Vanuatu,Entity_Venezuela,Entity_Vietnam,Entity_World,Entity_Yemen,Entity_Zambia,Entity_Zimbabwe
6014,2016,22.886465,3,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
71221,2010,10.112088,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
120599,1959,8.623334,12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15706,1973,25.110266,11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
119862,1983,0.719259,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119879,1984,-0.726492,12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
103694,1996,22.505539,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
131932,1969,26.557200,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
146867,2023,25.862974,12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [195]:
X_test

,Year,Monthly Average Temp,Month,Entity_Afghanistan,Entity_Albania,Entity_Algeria,Entity_American Samoa,Entity_Andorra,Entity_Angola,Entity_Anguilla,...,Entity_United States Virgin Islands,Entity_Uruguay,Entity_Uzbekistan,Entity_Vanuatu,Entity_Venezuela,Entity_Vietnam,Entity_World,Entity_Yemen,Entity_Zambia,Entity_Zimbabwe
50398,1974,8.713705,11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
115782,1983,27.948732,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
112125,2018,22.812632,10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
56427,1967,25.615232,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
97929,1940,23.996480,10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153472,1979,25.122467,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
68319,2023,25.625336,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
110372,1957,9.114661,9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
112369,1954,21.578516,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
ct_train = ColumnTransformer([
        ('somename', StandardScaler(), ['Monthly Average Temp'])
    ], remainder='passthrough')

ct_test = ColumnTransformer([
        ('somename', StandardScaler(), ['Monthly Average Temp'])
    ], remainder='passthrough')

X_train_scaled = ct_train.fit_transform(X_train)
X_test_scaled = ct_test.fit_transform(X_test)

'''
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
'''

ValueError: A given column is not a column of the dataframe

In [ ]:
X_train_scaled

array([[ 1.38860115,  0.55454338, -1.01372844, ..., -0.07170732,
        -0.07197248, -0.07153002],
       [ 1.14393047, -0.91205378, -1.30331574, ..., -0.07170732,
        -0.07197248, -0.07153002],
       [-0.93577025, -1.08297424,  1.59255725, ..., -0.07170732,
        -0.07197248, -0.07153002],
       ...,
       [-0.52798579,  0.97597212, -0.43455384, ..., -0.07170732,
        -0.07197248, -0.07153002],
       [ 1.67405027,  0.89626961,  1.59255725, ..., -0.07170732,
        -0.07197248, -0.07153002],
       [ 0.24680467, -0.01046768, -1.01372844, ..., -0.07170732,
        -0.07197248, -0.07153002]], shape=(159120, 198))

# Linear Regression

## Linear Regression: Training

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

In [ ]:
# Fit the linear regression model
linear_regression = LinearRegression()
linear_regression.fit(X_train_scaled, y_train)

# Make predictions
linear_regression_y_pred = linear_regression.predict(X_test_scaled)

## Linear Regression: Evaluation

In [ ]:
# Mean Squared Error
mse = mean_squared_error(y_test, linear_regression_y_pred)
print(f"Mean Squared Error: {mse}")

# R-squared
r2 = r2_score(y_test, linear_regression_y_pred)
print(f"R-squared: {r2}")

'''
accuracy = accuracy_score(y_test, linear_regression_y_pred)
print(f'Accuracy: {accuracy:.2f}')
'''

Mean Squared Error: 28.307315561204874
R-squared: 0.7322845776427029


"\naccuracy = accuracy_score(y_test, linear_regression_y_pred)\nprint(f'Accuracy: {accuracy:.2f}')\n"